IMDb 電影資料庫
電影評論數據集   
50000篇文字評論並且被標記為正或是負
我們試著用將影評訓練集和標籤來建立模型並利用影評測試集計算正評和負評的正確率

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
由上列網址下載數據集  並解壓縮到與 Sentiment Analysis.ipynb 同一個資料夾


In [2]:
! pip install pyprind

  Running setup.py bdist_wheel for pyprind: started
  Running setup.py bdist_wheel for pyprind: finished with status 'done'
  Stored in directory: C:\Users\mimur\AppData\Local\pip\Cache\wheels\cd\7b\7e\253b8df2dd2d8ea70d76870390300743f58a26317abb11e125
Successfully built pyprind


In [ ]:
import pyprind
import pandas as pd
import os


# unzipped movie dataset

basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
# 內有五萬筆資料
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
# 將'test', 'train' 資料 'pos', 'neg' 兩個類別都寫入 df
df.columns = ['review', 'sentiment']

0% [###############               ] 100% | ETA: 00:02:59

In [1]:
import pandas as pd
df.head(5)
# 資料還是有序的

NameError: name 'df' is not defined

In [30]:

import numpy as np
# 將資料順序打亂
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [31]:
df.head(10)

,review,sentiment
25957,election is a chinese mob movie or triads in t...,1
19386,i was just watching a forensic files marathon ...,0
31501,police story is a stunning series of set piece...,1
35365,dear readers the final battle between the rebe...,1
36753,i have seen the perfect son about three times ...,1
35358,a brilliant portrait of a traitor victor mclag...,1
4255,if ever a potential movie must ve sounded like...,1
27693,i d always wanted david duchovney to go into t...,1
13123,perhaps if only to laugh at the way my favorit...,0
433,even though the story is light the movie flows...,1


In [32]:
df.to_csv('./movie_data.csv', index=False, encoding='utf-8')
# 轉成CSV格式儲存

In [2]:
# 到這邊算是整理好資料的格式了  之後從這邊讀資料就好了
import pandas as pd

pd.read_csv('./movie_data.csv', encoding='utf-8')
df.head(10)



NameError: name 'df' is not defined

# Cleaning text data


In [34]:
df.loc[0, 'review'][-50:]


'and i suggest that you go see it before you judge '

In [35]:
# 清除一些html標籤和表情符號
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [36]:
preprocessor(df.loc[0, 'review'][-50:])


'and i suggest that you go see it before you judge '

In [37]:
preprocessor("</a>This :) is :( a test :-)!")


'this is a test :) :( :)'

In [38]:
df['review'] = df['review'].apply(preprocessor)


# Processing documents into tokens (中文則需要斷詞)


In [39]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

# 分割成單獨的字詞
def tokenizer(text):
    return text.split()

#　還原成原型
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [40]:
tokenizer('runners like running and thus they run')


['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [41]:
tokenizer_porter('runners like running and thus they run')


['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [42]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mimur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
from nltk.corpus import stopwords
# 停用字刪除 and, a...
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 用邏輯斯迴歸做分類

In [44]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
# 5 fold & grid-search 找最佳參數
# 可能要跑半個小時, 我把最佳參數挑出來  其他的先關掉了 
# n_jobs=1, 可能比較不會影響你們電腦其他東西運作

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
#               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer],

#               'clf__penalty': ['l1', 'l2'],
               'clf__penalty': ['l2'],
#               'clf__C': [1.0, 10.0, 100.0]},
                'clf__C': [10.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
#               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer],
               'vect__use_idf':[False],
               'vect__norm':[None],
#               'clf__penalty': ['l1', 'l2'],
               'clf__penalty': ['l2'],
#               'clf__C': [1.0, 10.0, 100.0]},
                'clf__C': [10.0]}
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1)

In [50]:
gs_lr_tfidf.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', '...01BF28>], 'vect__use_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l2'], 'clf__C': [10.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
     

In [51]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not

In [52]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.741


# Working with bigger data - online algorithms and out-of-core learning

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
next(stream_docs(path='./movie_data.csv'))


('election is a chinese mob movie or triads in this case every two years an election is held to decide on a new leader and at first it seems a toss up between big d tony leung ka fai or as i know him the other tony leung and lok simon yam who was judge in full contact though once lok wins big d refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader unlike any other asian film i watch featuring gangsters this one is not an action movie it has its bloody moments when necessary as in goodfellas but it s basically just a really effective drama there are a lot of characters which is really hard to keep track of but i think that plays into the craziness of it all a bit a 100 year old baton which is the symbol of power i mentioned before changes hands several times before things settle down and though it may appear that the film ends at the 65 or 70 minute mark there are still a couple big surprises waiting simon yam was my favorite character 

In [4]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [56]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

In [57]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


In [58]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.866


In [59]:
clf = clf.partial_fit(X_test, y_test)


# Serializing fitted scikit-learn estimators


In [60]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [61]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Writing movieclassifier/vectorizer.py


In [62]:
import os
os.chdir('movieclassifier')

In [63]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [64]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ['I love this movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], clf.predict_proba(X).max()*100))

Prediction: positive
Probability: 82.18%


# Setting up a SQLite database for data storage

In [65]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [66]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2015-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [67]:
print(results)


[('I love this movie', 1, '2017-07-08 14:23:00'), ('I disliked this movie', 0, '2017-07-08 14:23:00')]


# Developing a web application with Flask


In [69]:
!　pip install flask
